In [11]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [12]:
#opening the Assets and Liabilities of Commercial Bank data. Weekly estimated aggregated balance sheet for U.S. commercial banks,

csv_H8 = Path("Assets and Liabilities of Commercial Banks.csv")
as_Liab_df = pd.read_csv(csv_H8, skiprows=0)
as_Liab_df.columns = as_Liab_df.columns.str.strip().str.lower().str.replace(' ', '_')
as_Liab_df.dropna()


,series_description,"bank_credit,_all_commercial_banks,_not_seasonally_adjusted","securities_in_bank_credit,_all_commercial_banks,_not_seasonally_adjusted","treasury_and_agency_securities,_all_commercial_banks,_not_seasonally_adjusted","other_securities,_all_commercial_banks,_not_seasonally_adjusted","real_estate_loans,_all_commercial_banks,_not_seasonally_adjusted","total_assets,_all_commercial_banks,_not_seasonally_adjusted"
0,2017-12,12595204.2,3445892.7,2534829.1,911063.6,4292150.7,16764099.3
1,2018-01,12577899.7,3433981.2,2524045.1,909936.1,4300936.4,16711584.8
2,2018-02,12542097.2,3405207.2,2505051.8,900155.5,4304646.2,16684945.8
3,2018-03,12536604.0,3381190.4,2495684.9,885505.5,4311483.1,16594119.6
4,2018-04,12620814.1,3365089.9,2501785.6,863304.4,4324866.8,16575606.6
...,...,...,...,...,...,...,...
75,2024-03,17533757.6,5229598.3,4216355.9,1013242.4,5564668.2,23366770.6
76,2024-04,17527214.8,5178941.5,4171839.7,1007101.8,5580724.0,23238490.7
77,2024-05,17546914.5,5170648.4,4168777.4,1001871.1,5588652.3,23255908.0
78,2024-06,17646441.8,5220992.0,4222716.3,998275.7,5591222.7,23348220.2


In [13]:


csv_H6 = Path("Money Stock Measures.csv")
money_st_df = pd.read_csv(csv_H6, skiprows=0)
money_st_df.columns = money_st_df.columns.str.strip().str.lower().str.replace(' ', '_')
money_st_df.dropna()



,series_description,m1;_not_seasonally_adjusted,m2;_not_seasonally_adjusted,currency;_not_seasonally_adjusted,demand_deposits;_not_seasonally_adjusted,small-denomination_time_deposits_-_total;_not_seasonally_adjusted,retail_money_market_funds;_not_seasonally_adjusted,ira_and_keogh_accounts_at_depository_institutions;_not_seasonally_adjusted._last_4_obs_are_estimates.,ira_and_keogh_accounts_at_money_market_funds;_not_seasonally_adjusted._last_4_obs_are_estimates.,ira_and_keogh_accounts_-_total;_not_seasonally_adjusted._last_4_obs_are_estimates.,...,currency;_seasonally_adjusted,demand_deposits;_seasonally_adjusted,small-denomination_time_deposits_-_total;_seasonally_adjusted,retail_money_market_funds;_seasonally_adjusted,monetary_base;_currency_in_circulation;_not_seasonally_adjusted,monetary_base;_reserve_balances;_not_seasonally_adjusted,monetary_base;_total;_not_seasonally_adjusted,"reserves_of_depository_institutions,_total;_not_seasonally_adjusted",total_borrowings_from_the_federal_reserve;_not_seasonally_adjusted,"reserves_of_depository_institutions,_nonborrowed;_not_seasonally_adjusted"
0,2018-01,3651.9,13863.1,1528.7,1512.7,433.6,727.0,477.1,233.5,710.6,...,1534.6,1513.4,439.4,706.8,1610.2,2214.6,3824.8,2281.0,57.9,2281.0
1,2018-02,3566.1,13849.1,1538.2,1433.4,441.2,732.0,476.9,230.9,707.8,...,1543.7,1469.1,445.0,714.4,1616.3,2238.8,3855.1,2305.3,20.3,2305.2
2,2018-03,3688.4,14031.0,1555.3,1513.4,451.0,735.0,476.6,228.3,704.9,...,1552.5,1509.1,453.0,720.7,1633.6,2167.0,3800.6,2230.9,16.0,2230.9
3,2018-04,3698.1,14069.5,1564.9,1504.8,464.0,730.4,476.0,227.5,703.5,...,1561.1,1486.0,463.9,725.1,1640.9,2086.2,3727.1,2151.0,50.6,2151.0
4,2018-05,3655.7,13980.4,1575.9,1465.8,477.5,732.0,475.1,228.1,703.3,...,1571.3,1471.4,475.1,739.9,1652.3,2022.5,3674.8,2086.6,93.9,2086.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-01,18003.5,20822.3,2244.5,4966.5,1098.1,1720.6,466.1,550.3,1016.4,...,2253.3,4959.0,1106.5,1669.5,2336.7,3507.0,5843.7,3507.0,160073.0,3346.9
73,2024-02,17897.7,20746.9,2243.2,4934.1,1105.3,1743.9,461.4,555.0,1016.4,...,2253.2,4942.0,1112.8,1701.1,2329.2,3567.7,5896.9,3567.7,170206.8,3397.5
74,2024-03,18095.5,20976.0,2255.2,5060.0,1112.1,1768.4,456.7,559.7,1016.4,...,2256.2,5033.7,1117.5,1728.4,2339.9,3543.1,5883.0,3543.1,160502.2,3382.6
75,2024-04,18061.0,20960.7,2262.4,5197.7,1112.3,1787.4,454.6,561.8,1016.4,...,2258.1,5159.7,1114.9,1767.7,2345.7,3429.5,5775.2,3429.5,135937.9,3293.5


In [14]:
csv_G19 = Path("Consumer Credit.csv")
credit_Cr_df = pd.read_csv(csv_G19, skiprows=0)
credit_Cr_df.columns = credit_Cr_df.columns.str.strip().str.lower().str.replace(' ', '_')
credit_Cr_df.dropna()


,series_description,"percent_change_of_total_consumer_credit,_seasonally_adjusted_at_an_annual_rate","percent_change_of_total_revolving_consumer_credit,_seasonally_adjusted_at_an_annual_rate","percent_change_of_total_nonrevolving_consumer_credit,_seasonally_adjusted_at_an_annual_rate","total_consumer_credit_owned_and_securitized,_seasonally_adjusted_level","revolving_consumer_credit_owned_and_securitized,_seasonally_adjusted_level","nonrevolving_consumer_credit_owned_and_securitized,_seasonally_adjusted_level","total_consumer_credit_owned_and_securitized,_not_seasonally_adjusted_level","total_securitized_consumer_credit,_not_seasonally_adjusted_level","total_consumer_credit_owned_and_securitized_by_depository_institutions,_not_seasonally_adjusted_level",...,"nonrevolving_consumer_credit_owned_by_depository_institutions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_securitized_by_depository_institutions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_and_securitized_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_securitized_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_credit_unions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_federal_government,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_nonprofit_and_educational_institutions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_and_securitized_by_nonfinancial_business,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_nonfinancial_business,_not_seasonally_adjusted_flow,_monthly_rate"
0,2019-01,5.95,7.61,5.36,4026910.98,1060529.84,2966381.14,4016432.62,5226.17,1671715.49,...,4332.04,-248.04,-480.05,-480.05,0,662.57,26715.81,-311.72,95.87,95.87
1,2019-02,3.95,2.88,4.33,4040157.51,1063070.96,2977086.55,4000360.92,4990.44,1655238.11,...,124.09,-223.48,-2687.83,-2687.83,0,-777.69,5742.25,-265.23,-49.10,-49.10
2,2019-03,3.54,0.23,4.72,4052074.40,1063270.31,2988804.09,3994480.53,4731.99,1649257.98,...,2871.89,-245.65,-58.63,-58.63,0,481.85,840.92,-418.73,-67.28,-67.28
3,2019-04,5.16,7.80,4.22,4069488.87,1070185.63,2999303.24,4013774.09,4521.14,1664124.31,...,6712.60,-210.09,648.50,648.50,0,2835.56,796.40,-384.63,-27.60,-27.60
4,2019-05,3.82,3.17,4.05,4082449.85,1073016.09,3009433.75,4038943.87,4305.88,1681512.45,...,4917.22,-215.26,2325.09,2325.09,0,-1051.25,5040.43,-318.98,-19.58,-19.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2023-09,2.08,5.36,0.94,4991349.60,1294537.30,3696812.31,4962992.59,3841.17,2065206.23,...,296.36,425.61,2904.75,2904.75,0,1840.07,6177.72,338.09,28.42,28.42
57,2023-10,2.26,5.21,1.22,5000737.39,1300153.38,3700584.00,4972031.23,7976.39,2070634.63,...,-3334.49,4137.12,2619.65,2619.65,0,1004.45,-1066.71,-228.15,-14.47,-14.47
58,2023-11,4.38,13.35,1.22,5018975.81,1314617.56,3704358.25,5015150.47,12201.03,2111718.31,...,-3197.50,4226.59,2601.01,2601.01,0,-1955.00,359.32,-236.55,-21.65,-21.65
59,2023-12,1.13,3.83,0.17,5023699.47,1318812.16,3704887.30,5023699.47,16263.30,2116923.27,...,-14619.02,4064.45,2930.30,2930.30,0,-1225.09,-169.61,-91.63,-43.14,-43.14


In [28]:
merged_df = pd.merge(as_Liab_df, money_st_df, on='series_description', how='inner')
final_df = pd.merge(merged_df, credit_Cr_df, on='series_description', how='inner')
final_df.dropna()

,series_description,"bank_credit,_all_commercial_banks,_not_seasonally_adjusted","securities_in_bank_credit,_all_commercial_banks,_not_seasonally_adjusted","treasury_and_agency_securities,_all_commercial_banks,_not_seasonally_adjusted","other_securities,_all_commercial_banks,_not_seasonally_adjusted","real_estate_loans,_all_commercial_banks,_not_seasonally_adjusted","total_assets,_all_commercial_banks,_not_seasonally_adjusted",m1;_not_seasonally_adjusted,m2;_not_seasonally_adjusted,currency;_not_seasonally_adjusted,...,"nonrevolving_consumer_credit_owned_by_depository_institutions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_securitized_by_depository_institutions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_and_securitized_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_securitized_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_credit_unions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_federal_government,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_nonprofit_and_educational_institutions,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_and_securitized_by_nonfinancial_business,_not_seasonally_adjusted_flow,_monthly_rate","nonrevolving_consumer_credit_owned_by_nonfinancial_business,_not_seasonally_adjusted_flow,_monthly_rate"
0,2019-01,13182845.3,3544903.5,2715544.1,829359.5,4423221.8,16990230.5,3748.5,14423.8,1623.6,...,4332.04,-248.04,-480.05,-480.05,0,662.57,26715.81,-311.72,95.87,95.87
1,2019-02,13186105.6,3544991.1,2716072.0,828919.1,4425163.1,17045684.5,3705.1,14401.9,1628.5,...,124.09,-223.48,-2687.83,-2687.83,0,-777.69,5742.25,-265.23,-49.10,-49.10
2,2019-03,13221065.4,3559334.6,2732143.5,827191.1,4435195.9,17104931.2,3757.3,14572.5,1641.6,...,2871.89,-245.65,-58.63,-58.63,0,481.85,840.92,-418.73,-67.28,-67.28
3,2019-04,13307387.6,3607067.6,2769758.4,837309.2,4450201.8,17065992.8,3823.8,14623.0,1649.3,...,6712.60,-210.09,648.50,648.50,0,2835.56,796.40,-384.63,-27.60,-27.60
4,2019-05,13370639.4,3620906.3,2789156.5,831749.8,4477501.5,17130859.5,3791.3,14575.0,1655.6,...,4917.22,-215.26,2325.09,2325.09,0,-1051.25,5040.43,-318.98,-19.58,-19.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2023-09,17255552.4,5059419.2,4028562.9,1030856.3,5503144.9,22831239.0,18134.1,20668.7,2239.4,...,296.36,425.61,2904.75,2904.75,0,1840.07,6177.72,338.09,28.42,28.42
57,2023-10,17251037.5,5014583.8,3995048.8,1019534.9,5512329.9,22898808.5,18020.9,20627.9,2237.6,...,-3334.49,4137.12,2619.65,2619.65,0,1004.45,-1066.71,-228.15,-14.47,-14.47
58,2023-11,17304961.1,5039373.3,4025278.4,1014095.0,5525497.9,23098779.7,18016.7,20701.0,2241.0,...,-3197.50,4226.59,2601.01,2601.01,0,-1955.00,359.32,-236.55,-21.65,-21.65
59,2023-12,17436808.3,5101661.9,4085120.8,1016541.2,5536796.4,23326053.9,18113.3,20855.2,2247.7,...,-14619.02,4064.45,2930.30,2930.30,0,-1225.09,-169.61,-91.63,-43.14,-43.14


In [30]:
final_df.rename(columns={'series_description': 'date'}, inplace=True)

# Define the mapping from old column names to the new names you provided
column_mapping = {
    'bank_credit,_all_commercial_banks,_not_seasonally_adjusted': 'bank_credit',
    'securities_in_bank_credit,_all_commercial_banks,_not_seasonally_adjusted': 'securities_in_bank_credit',
    'treasury_and_agency_securities,_all_commercial_banks,_not_seasonally_adjusted': 'treasury_agency_securities',
    'other_securities,_all_commercial_banks,_not_seasonally_adjusted': 'other_securities',
    'real_estate_loans,_all_commercial_banks,_not_seasonally_adjusted': 'real_estate_loans',
    'total_assets,_all_commercial_banks,_not_seasonally_adjusted': 'total_assets',
    'currency;_seasonally_adjusted': 'currency_sa',
    'demand_deposits;_seasonally_adjusted': 'demand_deposits_sa',
    'monetary_base;_currency_in_circulation;_not_seasonally_adjusted': 'monetary_base_currency_nsa',
    'monetary_base;_total;_not_seasonally_adjusted': 'monetary_base_total_nsa',
    'percent_change_of_total_consumer_credit,_seasonally_adjusted_at_an_annual_rate': 'pct_change_consumer_credit_sa',
    'total_consumer_credit_owned_and_securitized,_seasonally_adjusted_level': 'consumer_credit_owned_securitized_sa',
    'total_consumer_credit_owned_by_nonfinancial_business,_not_seasonally_adjusted_level': 'consumer_credit_nonfinancial_owned_nsa',
    'total_consumer_credit_securitized_by_nonfinancial_business,_not_seasonally_adjusted_level': 'consumer_credit_nonfinancial_securitized_nsa',
    'total_consumer_credit_owned_by_credit_unions,_not_seasonally_adjusted_level': 'consumer_credit_credit_unions_owned_nsa',
}

# Rename the columns in final_df based on the mapping
final_df.rename(columns=column_mapping, inplace=True)

# Verify the result
# Define the list of columns you want to keep (those you provided)
columns_to_keep = [
    'date',
    'bank_credit',
    'securities_in_bank_credit',
    'treasury_agency_securities',
    'other_securities',
    'real_estate_loans',
    'total_assets',
    'currency_sa',
    'demand_deposits_sa',
    'monetary_base_currency_nsa',
    'monetary_base_total_nsa',
    'pct_change_consumer_credit_sa',
    'consumer_credit_owned_securitized_sa',
    'consumer_credit_nonfinancial_owned_nsa',
    'consumer_credit_nonfinancial_securitized_nsa',
    'consumer_credit_credit_unions_owned_nsa'
]

# Drop the columns that are not in the 'columns_to_keep' list
final_df = final_df[columns_to_keep]

# Verify the result
final_df.head()



,date,bank_credit,securities_in_bank_credit,treasury_agency_securities,other_securities,real_estate_loans,total_assets,currency_sa,demand_deposits_sa,monetary_base_currency_nsa,monetary_base_total_nsa,pct_change_consumer_credit_sa,consumer_credit_owned_securitized_sa,consumer_credit_nonfinancial_owned_nsa,consumer_credit_nonfinancial_securitized_nsa,consumer_credit_credit_unions_owned_nsa
0,2019-01,13182845.3,3544903.5,2715544.1,829359.5,4423221.8,16990230.5,1629.5,1481.0,1707.8,3346.9,5.95,4026910.98,36019.72,0,481454.25
1,2019-02,13186105.6,3544991.1,2716072.0,828919.1,4425163.1,17045684.5,1635.3,1488.2,1708.2,3353.5,3.95,4040157.51,35518.03,0,480157.91
2,2019-03,13221065.4,3559334.6,2732143.5,827191.1,4435195.9,17104931.2,1640.3,1462.1,1720.2,3381.5,3.54,4052074.40,35197.38,0,480787.97
3,2019-04,13307387.6,3607067.6,2769758.4,837309.2,4450201.8,17065992.8,1646.3,1481.0,1727.2,3286.7,5.16,4069488.87,35109.26,0,483632.30
4,2019-05,13370639.4,3620906.3,2789156.5,831749.8,4477501.5,17130859.5,1650.9,1484.4,1733.2,3244.5,3.82,4082449.85,35208.25,0,483653.81


In [31]:

final_df['date'] = pd.to_datetime(final_df['date'], format='%Y-%m')

final_df

C:\Users\AndBe\AppData\Local\Temp\ipykernel_20004\4215680771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['date'] = pd.to_datetime(final_df['date'], format='%Y-%m')


,date,bank_credit,securities_in_bank_credit,treasury_agency_securities,other_securities,real_estate_loans,total_assets,currency_sa,demand_deposits_sa,monetary_base_currency_nsa,monetary_base_total_nsa,pct_change_consumer_credit_sa,consumer_credit_owned_securitized_sa,consumer_credit_nonfinancial_owned_nsa,consumer_credit_nonfinancial_securitized_nsa,consumer_credit_credit_unions_owned_nsa
0,2019-01-01,13182845.3,3544903.5,2715544.1,829359.5,4423221.8,16990230.5,1629.5,1481.0,1707.8,3346.9,5.95,4026910.98,36019.72,0,481454.25
1,2019-02-01,13186105.6,3544991.1,2716072.0,828919.1,4425163.1,17045684.5,1635.3,1488.2,1708.2,3353.5,3.95,4040157.51,35518.03,0,480157.91
2,2019-03-01,13221065.4,3559334.6,2732143.5,827191.1,4435195.9,17104931.2,1640.3,1462.1,1720.2,3381.5,3.54,4052074.40,35197.38,0,480787.97
3,2019-04-01,13307387.6,3607067.6,2769758.4,837309.2,4450201.8,17065992.8,1646.3,1481.0,1727.2,3286.7,5.16,4069488.87,35109.26,0,483632.30
4,2019-05-01,13370639.4,3620906.3,2789156.5,831749.8,4477501.5,17130859.5,1650.9,1484.4,1733.2,3244.5,3.82,4082449.85,35208.25,0,483653.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2023-09-01,17255552.4,5059419.2,4028562.9,1030856.3,5503144.9,22831239.0,2241.5,4929.1,2327.7,5567.3,2.08,4991349.60,35087.52,0,661435.27
57,2023-10-01,17251037.5,5014583.8,3995048.8,1019534.9,5512329.9,22898808.5,2243.6,4950.7,2324.9,5601.3,2.26,5000737.39,35109.07,0,663516.21
58,2023-11-01,17304961.1,5039373.3,4025278.4,1014095.0,5525497.9,23098779.7,2246.6,4949.3,2328.0,5731.5,4.38,5018975.81,35548.86,0,662280.29
59,2023-12-01,17436808.3,5101661.9,4085120.8,1016541.2,5536796.4,23326053.9,2249.8,5008.7,2335.4,5827.4,1.13,5023699.47,35777.78,0,662586.72


In [34]:
final_df.to_csv('output data/final_data.csv', index=False)


In [18]:
numeric_df = final_df.select_dtypes(include='number')
numeric_df = numeric_df.dropna()  


In [19]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)


scaled_df = pd.DataFrame(scaled_data, columns=numeric_df.columns, index=numeric_df.index)


In [20]:
import hvplot.pandas


from sklearn.cluster import KMeans

inertia = []

k = list(range(1, 11))

#Loop through the list of k values, fit a K-Means model
for i in k:
    k_model = KMeans(n_clusters=i, random_state=2)
    k_model.fit(scaled_df)
    inertia.append(k_model.inertia_)

# DataFrame to store the elbow data for plotting
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

df_elbow.hvplot.line(x='k', y='inertia', title='Elbow Curve', xlabel='Number of clusters (k)', ylabel='Inertia')


:Curve   [k]   (inertia)

In [12]:
model = KMeans(n_clusters=4, random_state=1)


model.fit(scaled_df)
k_lower = model.predict(scaled_df)

home_sales_predictions_df = scaled_df.copy()
#class column with the labels to the home_sales_predictions_df DataFrame
home_sales_predictions_df['clusters_lower'] = k_lower

In [13]:
print(home_sales_predictions_df.columns)

Index(['bank_credit,_all_commercial_banks,_not_seasonally_adjusted',
       'securities_in_bank_credit,_all_commercial_banks,_not_seasonally_adjusted',
       'treasury_and_agency_securities,_all_commercial_banks,_not_seasonally_adjusted',
       'other_securities,_all_commercial_banks,_not_seasonally_adjusted',
       'real_estate_loans,_all_commercial_banks,_not_seasonally_adjusted',
       'total_assets,_all_commercial_banks,_not_seasonally_adjusted',
       'm1;_not_seasonally_adjusted', 'm2;_not_seasonally_adjusted',
       'currency;_not_seasonally_adjusted',
       'demand_deposits;_not_seasonally_adjusted',
       ...
       'nonrevolving_consumer_credit_securitized_by_depository_institutions,_not_seasonally_adjusted_flow,_monthly_rate',
       'nonrevolving_consumer_credit_owned_and_securitized_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate',
       'nonrevolving_consumer_credit_owned_by_finance_companies,_not_seasonally_adjusted_flow,_monthly_rate',
       '

In [15]:
home_sales_predictions_df.hvplot.scatter(
    x="demand_deposits;_not_seasonally_adjusted",
    y="bank_credit,_all_commercial_banks,_not_seasonally_adjusted",
    by="clusters_lower"
).opts(yformatter="%.0f")

:NdOverlay   [clusters_lower]
   :Scatter   [demand_deposits;_not_seasonally_adjusted]   (bank_credit,_all_commercial_banks,_not_seasonally_adjusted)

In [14]:
#washing the data from time variable